# Compras Governamentais - MIRA

Acessando os dados das nfe do portal da transparência da CGU.

In [1]:
import duckdb, pandas as pd

from IPython.display import display
import numpy as np
from decimal import Decimal

pd.set_option("display.float_format", lambda x: f"{x:_.2f}") 
def underscore_fmt(x):
    if pd.isna(x): return ""
    if isinstance(x, (int, np.integer)):   return f"{x:_d}"
    if isinstance(x, (float, np.floating)): return f"{x:_.0f}"
    if isinstance(x, Decimal):             return f"{x:_.0f}"
    return x

In [2]:

db = duckdb.connect(database='../data/outputs/mgi/dw.db')

In [3]:

for e in ("spatial","httpfs","json","excel"): db.install_extension(e); db.load_extension(e)
for e in ("zipfs",): db.install_extension(e, repository="community"); db.load_extension(e)

In [4]:
db.execute("""ATTACH '../data/outputs/cgu/nfe.db' as nfe""")
db.execute("""select * from nfe.cgu.notas limit 2""").df()

,chave_acesso,modelo,serie,numero,natureza_operacao,timestamp_emissao,data_emissao,mes_emissao,ano_emissao,ultimo_evento,...,orgao_destinatario,destinatario,nome_destinatario,uf_destinatario,indicador_ie_destinatario,destino_operacao,indicador_consumidor_final,indicador_presenca_comprador,valor_nota_fiscal,duplicada
0,42220107432517001847550030005499571000224173,55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 ...,3,549957,Outra saida merc./prest.serv. nao especif.,2022-01-01 00:02:22,2022-01-01,1,2022,Autorização de Uso,...,Sem informação,00394460043975,MINISTERIO DA ECONOMIA,DF,NÃO CONTRIBUINTE,2 - OPERAÇÃO INTERESTADUAL,1 - CONSUMIDOR FINAL,0 - NÃO SE APLICA,62.14,False
1,42220107432517001847550030005499821000227345,55 - NF-E EMITIDA EM SUBSTITUIÇÃO AO MODELO 1 ...,3,549982,Outra saida merc./prest.serv. nao especif.,2022-01-01 00:03:59,2022-01-01,1,2022,Autorização de Uso,...,"Instituto Federal de Educação, Ciência e Tecno...",10779511000298,"INSTITUTO FEDERAL DE EDUCACAO, CIENCIA E TECNO...",RJ,CONTRIBUINTE ISENTO,2 - OPERAÇÃO INTERESTADUAL,1 - CONSUMIDOR FINAL,0 - NÃO SE APLICA,224.42,False


In [5]:
canceladas = db.execute("""SELECT chave_acesso from nfe.cgu.eventos where evento = 'Cancelamento da NF-e'""").df()
canceladas.head(2)

,chave_acesso
0,11220103552842000144550000007133711021401139
1,11220103983300000550550010000486931172800086


Note que há notas que não são de compras, mas apenas de transferência dentro do mesmo órgão.

In [6]:
db.execute("select chave_acesso, natureza_operacao, emitente, nome_emitente, destinatario, nome_destinatario from nfe.cgu.notas where emitente = destinatario").df()

,chave_acesso,natureza_operacao,emitente,nome_emitente,destinatario,nome_destinatario
0,31220110648539000296550010000039731554575770,Transferencia de produtos para serem comercial...,10648539000296,"INSTITUTO FEDERAL DE EDUC, CIENCIA TEC. SUL DE MG",10648539000296,IFSULDEMINAS - CAMPUS MUZAMBINHO (1440)
1,31220110648539000296550010000039741177514000,Transferencia de produtos para serem comercial...,10648539000296,"INSTITUTO FEDERAL DE EDUC, CIENCIA TEC. SUL DE MG",10648539000296,IFSULDEMINAS - CAMPUS MUZAMBINHO (1440)
2,15220134621748000638550010000060251000398021,TRANSFERÊNCIA PARA COMERCIALIZACAO,34621748000638,EDITORA DA UFPA,34621748000638,UNIVERSIDADE FEDERAL DO PARA - EDITORA UFPA
3,53220100038174000496550010000120881001461526,"DOACAO, BONIFICACAO OU BRINDE",00038174000496,FUNDACAO UNIVERSIDADE DE BRASILIA,00038174000496,FUNDACAO UNIVERSIDADE DE BRASILIA
4,31220110648539000296550010000039751325780273,Transferencia de produtos para serem comercial...,10648539000296,"INSTITUTO FEDERAL DE EDUC, CIENCIA TEC. SUL DE MG",10648539000296,IFSULDEMINAS - CAMPUS MUZAMBINHO (1440)
...,...,...,...,...,...,...
4569,31241210648539000296550010000061021136455891,Transferencia de produtos para serem comercial...,10648539000296,"INSTITUTO FEDERAL DE EDUC, CIENCIA TEC. SUL DE MG",10648539000296,IFSULDEMINAS - CAMPUS MUZAMBINHO (1440)
4570,31241217217985004282550010000414321000228601,INDUST FEITA POR TERCEIROS,17217985004282,UNIVERSIDADE FEDERAL DE MINAS GERAIS,17217985004282,UNIVERSIDADE FEDERAL DE MINAS GERAIS
4571,31241217217985004282550010000414331000228617,"ENTRADA DE BONIFICACAO, DOACAO OU BRINDE",17217985004282,UNIVERSIDADE FEDERAL DE MINAS GERAIS,17217985004282,UNIVERSIDADE FEDERAL DE MINAS GERAIS
4572,31241210648539000296550010000061041276387656,Transferencia de produtos para serem comercial...,10648539000296,"INSTITUTO FEDERAL DE EDUC, CIENCIA TEC. SUL DE MG",10648539000296,IFSULDEMINAS - CAMPUS MUZAMBINHO (1440)


Os primeiros 8 dígitos do CNPJ são a raiz que identifica pessoas jurídicas de mesma empresa.

In [7]:
db.execute("select chave_acesso, natureza_operacao, emitente, nome_emitente, destinatario, nome_destinatario from nfe.cgu.notas where left(emitente,8) = left(destinatario,8)").df()

,chave_acesso,natureza_operacao,emitente,nome_emitente,destinatario,nome_destinatario
0,31220110648539000296550010000039731554575770,Transferencia de produtos para serem comercial...,10648539000296,"INSTITUTO FEDERAL DE EDUC, CIENCIA TEC. SUL DE MG",10648539000296,IFSULDEMINAS - CAMPUS MUZAMBINHO (1440)
1,43220187958674000181558900383383291813921498,Saídas Diversas,00662270000915,INST NAC METROL QUALIDADE E TECNOLOGIA INMETRO,00662270000320,"INSTITUTO NACIONAL DE METROLOGIA, QUALIDADE E ..."
2,33220142498675000152558900041721101288054913,Outras saídas de mercadorias ou prestações de ...,00662270000320,'INSTITUTO NACIONAL DE METROLOGIA,00662270000915,INST NACIONAL DE METROLOGIA QUALIDADE INMETRO
3,31220110648539000296550010000039741177514000,Transferencia de produtos para serem comercial...,10648539000296,"INSTITUTO FEDERAL DE EDUC, CIENCIA TEC. SUL DE MG",10648539000296,IFSULDEMINAS - CAMPUS MUZAMBINHO (1440)
4,31220110648539000105550010000000531060950904,DEVOLUÇÃO BEM ATIVO IMOBILIZADO,10648539000105,INSTITUTO FEDERAL DE EDUCACAO CIENCIA E TECNOL...,10648539000296,IFSULDEMINAS - CAMPUS MUZAMBINHO (1440)
...,...,...,...,...,...,...
7535,31241217217985004282550010000414321000228601,INDUST FEITA POR TERCEIROS,17217985004282,UNIVERSIDADE FEDERAL DE MINAS GERAIS,17217985004282,UNIVERSIDADE FEDERAL DE MINAS GERAIS
7536,31241217217985004282550010000414331000228617,"ENTRADA DE BONIFICACAO, DOACAO OU BRINDE",17217985004282,UNIVERSIDADE FEDERAL DE MINAS GERAIS,17217985004282,UNIVERSIDADE FEDERAL DE MINAS GERAIS
7537,53241200038174001468550010000005721001690177,TRANSFERENCIA PRODUTO SAIDA,00038174001468,FUNDACAO UNIVERSIDADE DE BRASILIA,00038174001972,FUNDACAO UNIVERSIDADE DE BRASILIA - LIVRARIA
7538,31241210648539000296550010000061041276387656,Transferencia de produtos para serem comercial...,10648539000296,"INSTITUTO FEDERAL DE EDUC, CIENCIA TEC. SUL DE MG",10648539000296,IFSULDEMINAS - CAMPUS MUZAMBINHO (1440)


## Filtrando a base CGU

In [8]:
compras_sql = """
           SELECT * FROM nfe.cgu.notas where ultimo_evento not in (
                'Cancelamento da NF-e', 
                'Manifestação do destinatário - Operação não realizada', 
                'Manifestação do destinatário - Desconhecimento da operação')
            AND chave_acesso NOT IN (SELECT chave_acesso from nfe.cgu.eventos where evento = 'Cancelamento da NF-e')
            AND chave_acesso NOT IN (SELECT chave_acesso from nfe.cgu.notas where left(emitente,8) = left(destinatario,8))
"""
db.execute("CREATE SCHEMA IF NOT EXISTS mgi")
db.execute(f"CREATE OR REPLACE TABLE mgi.notas AS {compras_sql}").df()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Count
0,4983258


In [9]:
db.query("SELECT ano_emissao, COUNT(*) as notas, SUM(valor_nota_fiscal) as valor_total FROM mgi.notas group by all").df()

,ano_emissao,notas,valor_total
0,2022,1639023,75_617_577_515.53
1,2023,1715084,71_704_876_669.73
2,2024,1629151,67_102_506_351.88


Não exatamente o que aparece no portal da transparência, mas é o que temos no momento.

In [10]:
db.execute("describe nfe.cgu.itens").df()

,column_name,column_type,null,key,default,extra
0,chave_acesso,VARCHAR,YES,None,None,None
1,timestamp_emissao,TIMESTAMP,YES,None,None,None
2,data_emissao,DATE,YES,None,None,None
3,mes_emissao,INTEGER,YES,None,None,None
4,ano_emissao,INTEGER,YES,None,None,None
5,uf_emitente,"ENUM('AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF',...",YES,None,None,None
6,uf_destinatario,"ENUM('AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF',...",YES,None,None,None
7,emitente,VARCHAR,YES,None,None,None
8,nome_emitente,VARCHAR,YES,None,None,None
9,destinatario,VARCHAR,YES,None,None,None


In [11]:
itens_sql = """
    SELECT * from nfe.cgu.itens where chave_acesso IN (SELECT chave_acesso FROM mgi.notas)
"""
display(db.execute(f"CREATE OR REPLACE TABLE mgi.itens AS {itens_sql}").df().style.format(underscore_fmt))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Count
0,18_021_620


In [12]:
db.query("SELECT ano_emissao, COUNT(*) as itens, SUM(valor_item) as valor_total FROM mgi.itens group by all").df()

,ano_emissao,itens,valor_total
0,2022,5492405,79_786_643_934.46
1,2023,6023240,80_817_805_443.94
2,2024,6505975,77_790_262_279.47


In [13]:
diferentes = db.query("""
SELECT
  i.chave_acesso,
  SUM(i.valor_item)         AS valor_itens,
  MAX(n.valor_nota_fiscal)  AS valor_nota,
  CASE 
    WHEN SUM(i.valor_item) = 0 THEN NULL
    ELSE 100.0 * (SUM(i.valor_item) - MAX(n.valor_nota_fiscal)) / SUM(i.valor_item)
  END AS diff_pct
FROM mgi.itens i
JOIN mgi.notas n USING (chave_acesso)
GROUP BY i.chave_acesso
HAVING ABS(SUM(i.valor_item) - MAX(n.valor_nota_fiscal)) > 0.01
ORDER BY i.chave_acesso
""").df()
diferentes

,chave_acesso,valor_itens,valor_nota,diff_pct
0,11220101055747000100550010000270531548085752,87.10,79.08,9.21
1,11220102436230000123550010000033411264818401,104.00,95.12,8.54
2,11220102436230000123550010000033421531465693,1_001.00,965.47,3.55
3,11220102436230000123550010000033461668287790,1_042.00,1_031.58,1.00
4,11220102436230000123550010000033581927160082,4_170.00,4_167.91,0.05
...,...,...,...,...
727367,53241272591043000108550010000041971009593958,1_750.00,1_563.45,10.66
727368,53241272591043000108550010000041991009610660,750.00,670.05,10.66
727369,53241275315333006140550010018329491758422969,321.99,321.94,0.02
727370,53241275315333006140550010018355301758499198,309.76,309.86,-0.03


In [14]:
diferentes.describe()

,valor_itens,valor_nota,diff_pct
count,727_372.00,727_372.00,686_147.00
mean,99_828.99,66_867.60,-419.95
std,1_910_251.41,1_228_095.36,323_402.87
min,0.00,0.00,-267_554_500.00
25%,384.94,376.61,-1.76
50%,1_682.80,1_585.00,0.00
75%,9_241.80,8_750.00,8.67
max,338_990_778.36,215_831_701.43,100.00


In [15]:
# Top 20 most negative diff_pct rows (excluding NULLs)
db.query("""
WITH agreg AS (
  SELECT
    i.chave_acesso,
    SUM(i.valor_item)        AS valor_itens,
    MAX(n.valor_nota_fiscal) AS valor_nota
  FROM mgi.itens i
  JOIN mgi.notas n USING (chave_acesso)
  GROUP BY 1
),
calc AS (
  SELECT
    chave_acesso,
    valor_itens,
    valor_nota,
    CASE WHEN valor_itens = 0 THEN NULL
         ELSE 100.0 * (valor_itens - valor_nota) / NULLIF(valor_itens, 0)
    END AS diff_pct
  FROM agreg
)
SELECT *
FROM calc
WHERE diff_pct IS NOT NULL
ORDER BY diff_pct ASC
LIMIT 20
""").df()

,chave_acesso,valor_itens,valor_nota,diff_pct
0,35241001382559000196550010000568791453969473,0.01,26_755.46,-267_554_500.00
1,33220267405936000173550030007312531090919160,0.01,1_150.86,-11_508_500.00
2,33231242498675000152558900082727141232845850,0.08,4_500.08,-5_625_000.00
3,33241042498675000152558900095996841388073319,0.03,1_074.03,-3_580_000.00
4,33241042498675000152558900096004241456950049,0.06,537.06,-895_000.00
5,33241042498675000152558900096000581631737660,0.09,537.09,-596_666.67
6,43231091967067000155550000004604271397197016,0.01,35.52,-355_100.00
7,35220859231530000193550010000963991554476272,0.06,115.71,-192_750.00
8,53241128843315000110550010000022981105953029,0.70,1_100.70,-157_142.86
9,33231242498675000152558900082646331199348194,0.80,1_000.80,-125_000.00


In [16]:
db.execute("select chave_acesso, timestamp_emissao, descricao_produto, ncm, valor_item FROM mgi.itens where chave_acesso = '35241001382559000196550010000568791453969473'").df()

,chave_acesso,timestamp_emissao,descricao_produto,ncm,valor_item
0,35241001382559000196550010000568791453969473,2024-10-25 13:10:00,ANALISADOR DE TEOR DE FAME EM COMBUSTIVEL DE A...,90278999,0.01


In [17]:
db.execute("select chave_acesso, timestamp_emissao, valor_nota_fiscal FROM mgi.notas where chave_acesso = '35241001382559000196550010000568791453969473'").df()

,chave_acesso,timestamp_emissao,valor_nota_fiscal
0,35241001382559000196550010000568791453969473,2024-10-25 13:10:00,26_755.46


Em geral, o valor_nota é menor que o valor_itens, mas há excecões 

In [18]:
db.query("""
SELECT
  i.chave_acesso,
  SUM(i.valor_item)         AS valor_itens,
  MAX(n.valor_nota_fiscal)  AS valor_nota
FROM mgi.itens i
JOIN mgi.notas n USING (chave_acesso)
where i.chave_acesso = '33221200776574001390550050329327331446711857'
GROUP BY i.chave_acesso
HAVING ABS(SUM(i.valor_item) - MAX(n.valor_nota_fiscal)) > 0.01
ORDER BY i.chave_acesso
""").df()

,chave_acesso,valor_itens,valor_nota
0,33221200776574001390550050329327331446711857,3_467.49,3_427.73


O valor do item não bate com a nota porque o item não está considerando valores de frete e desconto como visto abaixo:


![](./item_vs_nota.png)

In [19]:
db.execute("""
SELECT
  i.chave_acesso,
  SUM(i.valor_item)         AS valor_itens,
  MAX(n.valor_nota_fiscal)  AS valor_nota,
  100.0 * (SUM(i.valor_item) - MAX(n.valor_nota_fiscal)) / NULLIF(SUM(i.valor_item), 0) AS diff_pct
FROM mgi.itens i
JOIN mgi.notas n USING (chave_acesso)
GROUP BY i.chave_acesso
HAVING ABS( (SUM(i.valor_item) - MAX(n.valor_nota_fiscal)) / NULLIF(SUM(i.valor_item), 0) ) > 0.10
ORDER BY diff_pct DESC
           """).df()

,chave_acesso,valor_itens,valor_nota,diff_pct
0,33230700323090000313550020000025031283928813,17_084.10,0.00,100.00
1,41230713805697000110550010000317511328870283,136.70,0.00,100.00
2,25230816713807000101550010000031911947908580,149.45,0.00,100.00
3,27240712200192000169558900087794651946555809,1_210.12,0.00,100.00
4,43230540223106000179550010000058501000142758,35.87,0.00,100.00
...,...,...,...,...
195795,33241042498675000152558900096004241456950049,0.06,537.06,-895_000.00
195796,33241042498675000152558900095996841388073319,0.03,1_074.03,-3_580_000.00
195797,33231242498675000152558900082727141232845850,0.08,4_500.08,-5_625_000.00
195798,33220267405936000173550030007312531090919160,0.01,1_150.86,-11_508_500.00


In [20]:
196118/4990754 * 100.0

3.929626665630083

4% das notas tem valor_itens mais de 10% diferente do valor_nota, 2% mais de 20% diferente e 1% mais de 30% diferente. Vamos usar como base mgi.itens mas não estamos filtrando onde valor_item é diferente de valor_nota_fiscal.


### NCM

In [21]:
db.execute("""
    CREATE OR REPLACE TABLE mgi.ncm AS
    SELECT
        unnest."Codigo" AS ncm,
        unnest."Descricao" AS ncm_nome,
        strptime(unnest."Data_Inicio", '%d/%m/%Y')::date AS data_inicio
    FROM
        read_json('https://portalunico.siscomex.gov.br/classif/api/publico/nomenclatura/download/json'),
        UNNEST(Nomenclaturas) AS unnest
        """).df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Count
0,15141


In [22]:
db.query("FROM mgi.ncm").df()

,ncm,ncm_nome,data_inicio
0,01,Animais vivos.,2022-04-01
1,01.01,"Cavalos, asininos e muares, vivos.",2022-04-01
2,0101.2,- Cavalos:,2022-04-01
3,0101.21.00,-- Reprodutores de raça pura,2022-04-01
4,0101.29.00,-- Outros,2022-04-01
...,...,...,...
15136,9705.31.00,-- Com mais de 100 anos,2022-04-01
15137,9705.39.00,-- Outras,2022-04-01
15138,97.06,Antiguidades com mais de 100 anos.,2022-04-01
15139,9706.10.00,- Com mais de 250 anos,2022-04-01


de_ncm_para_scn128 no IBGE: https://ftp.ibge.gov.br/Contas_Nacionais/Sistema_de_Contas_Nacionais/Tradutores/TradutorNCM.xls

Ver: https://www.ibge.gov.br/estatisticas/economicas/servicos/9052-sistema-de-contas-nacionais-brasil.html?=&t=notas-tecnicas

In [23]:
db.execute("""
    CREATE OR REPLACE TABLE mgi.de_ncm_para_scn AS
    SELECT
        *
    FROM
        read_parquet('../data/inputs/mgi/de_ncm_para_scn128.parquet')
        """).df()

,Count
0,11875


In [24]:
db.execute("SELECT * FROM mgi.de_ncm_para_scn").df().head(3)

,NCM_COD,NCM_NOME,SCN128_COD,SCN128_DESC
0,01011010,Cavalos reprodutores de raça pura,01921,"Bovinos e outros animais vivos, prods. animal,..."
1,01011090,"Animais da espécie asinina/muar, reprodutores,...",01921,"Bovinos e outros animais vivos, prods. animal,..."
2,01011100,"Cavalos reprodutores, de raça pura",01921,"Bovinos e outros animais vivos, prods. animal,..."


In [25]:
db.execute("""
    CREATE OR REPLACE TABLE mgi.scn AS
    SELECT
        *
    FROM
        read_parquet('../data/inputs/mgi/scn.parquet')       
        """)
db.execute("""
           SELECT *
           FROM mgi.scn
""").df()

,SCN128_COD,SCN128_NOME,SCN67_COD,SCN67_NOME
0,01911,"Arroz, trigo e outros cereais",0191,"Agricultura, inclusive o apoio à agricultura e..."
1,01912,Milho em grão,0191,"Agricultura, inclusive o apoio à agricultura e..."
2,01913,"Algodão herbáceo, outras fibras da lav. tempor...",0191,"Agricultura, inclusive o apoio à agricultura e..."
3,01914,Cana-de-açúcar,0191,"Agricultura, inclusive o apoio à agricultura e..."
4,01915,Soja em grão,0191,"Agricultura, inclusive o apoio à agricultura e..."
...,...,...,...,...
123,90801,"Serviços de artes, cultura, esporte e recreação",9080,"Atividades artísticas, criativas e de espetáculos"
124,94801,"Organizações patronais, sindicais e outros ser...",9480,Organizações associativas e outros serviços pe...
125,94802,"Manutenção de computadores, telefones e objeto...",9480,Organizações associativas e outros serviços pe...
126,94803,Serviços pessoais,9480,Organizações associativas e outros serviços pe...


### CFOP

A tabela CFOP está em: [https://www.nfe.fazenda.gov.br > Documentos > Diversos](https://www.nfe.fazenda.gov.br/portal/listaConteudo.aspx?tipoConteudo=/NJarYc9nus=)

`https://www.nfe.fazenda.gov.br/portal/exibirArquivo.aspx?conteudo=UGAhInpwpPw=`

Mas a fonte mais completa de informações sobre CFOP está em: https://www.confaz.fazenda.gov.br/legislacao/ajustes/sinief/cfop_cvsn_70_vigente

Geramos uma tabela a partir do html existente em: https://www.lefisc.com.br/regulamentos/ricmsRR/anexoIX.asp



In [26]:
db.execute("""
           DROP SCHEMA IF EXISTS raw CASCADE;
           CREATE OR REPLACE SCHEMA raw
""")

In [27]:

db.execute("""
           create or replace table raw.cfop as 
           select
           REGEXP_REPLACE(LEFT(CFOP::VARCHAR, 5), '[^0-9]', '', 'g') AS CFOP,
           * EXCLUDE(CFOP)
           from read_xlsx('../data/inputs/rfb_cnpj/IT 2023.002-v1.00_Tabela_CFOP_Vigência_20230424.xlsx')""").df()

,Count
0,619


In [28]:
db.query("SELECT * from raw.cfop limit 3").df()

,CFOP,Início de vigência,Fim de vigência,indNFe,indComunica,indTransp,indDevol,indRetor,indAnula,indRemes,indComb
0,1101,2006-01-01,NaN,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,1102,2006-01-01,NaN,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,1111,2006-01-01,NaN,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [29]:
db.execute (
   """
CREATE OR REPLACE TABLE mgi.cfop AS
-- 1) Texto e aplicação (CSV)
WITH cfop_texto AS (
  SELECT
    REGEXP_REPLACE(LEFT(CFOP::VARCHAR, 5), '[^0-9]', '', 'g') AS cfop,
    * EXCLUDE (CFOP)
  FROM read_csv('../data/inputs/rfb_cnpj/cfop_anexoix.csv',
                header=true, delim=';', auto_type_candidates=['VARCHAR'])
),

-- 2) Base do XLSX com CFOP limpo
cfop_flags_base AS (
  SELECT
    REGEXP_REPLACE(LEFT(CFOP::VARCHAR, 5), '[^0-9]', '', 'g') AS cfop,
    RIGHT(REGEXP_REPLACE(LEFT(CFOP::VARCHAR, 5), '[^0-9]', '', 'g')::VARCHAR, 3) AS codigo_operacao,
    "Início de vigência" AS vigencia_inicio,
    "Fim de vigência"    AS vigencia_fim,
    indNFe, indComunica, indTransp, indDevol, indRetor, indAnula, indRemes, indComb
  FROM read_xlsx('../data/inputs/rfb_cnpj/IT 2023.002-v1.00_Tabela_CFOP_Vigência_20230424.xlsx')
),

-- 3) Campos derivados da base XLSX
cfop_flags_derivados AS (
  SELECT
    cfop,
    LEFT(cfop::VARCHAR, 1) AS codigo_fluxo,
    CASE
      WHEN LEFT(cfop::VARCHAR, 1) IN ('1','2','3') THEN 'entrada'
      WHEN LEFT(cfop::VARCHAR, 1) IN ('5','6','7') THEN 'saida'
    END AS fluxo_sentido,
    CASE
      WHEN LEFT(cfop::VARCHAR, 1) IN ('1','5') THEN 'intra-estadual'
      WHEN LEFT(cfop::VARCHAR, 1) IN ('2','6') THEN 'interestadual'
      WHEN LEFT(cfop::VARCHAR, 1) IN ('3','7') THEN 'internacional'
    END AS fluxo_abrangencia,
    codigo_operacao,
    vigencia_inicio,
    vigencia_fim,
    COALESCE(indNFe::BOOLEAN,      FALSE) AS indNFe,
    COALESCE(indComunica::BOOLEAN, FALSE) AS indComunica, -- consumo de serviços de comunicação sujeito a ICMS -- consumo intermediário
    COALESCE(indTransp::BOOLEAN,   FALSE) AS indTransp, --consumo de serviços de transporte sujeito a ICMS -- consumo intermediário
    COALESCE(indDevol::BOOLEAN,    FALSE) AS indDevol, -- devolução de mercadoria
    COALESCE(indRetor::BOOLEAN,    FALSE) AS indRetor, -- retorno de mercadoria (enviada provavelmente em remessa anterior)
    COALESCE(indAnula::BOOLEAN,    FALSE) AS indAnula, -- anulação de operação (como a devolução, mas sem devolução física de mercadoria)
    COALESCE(indRemes::BOOLEAN,    FALSE) AS indRemes, -- remessa de mercadoria (normalmente não envolve transferência de propriedade)
    COALESCE(indComb::BOOLEAN,     FALSE) AS indComb, -- consumo de combustível - consumo intermediário
    (LEFT(codigo_operacao, 2) = '60')                                  AS indTrib, -- operação meramente tributária (operação de crédito de ICMS, por exemplo_
    (codigo_operacao IN ('551','552','553','554','555'))               AS indFBKF -- operação que altera a Formação Bruta de Capital Fixo (FBKF)
  FROM cfop_flags_base
)

-- 4) Junção final
SELECT
  COALESCE(cfop_texto.cfop, cfop_flags_derivados.cfop) AS cfop,
  cfop_texto.descricao,
  cfop_texto.aplicacao,
  cfop_flags_derivados.codigo_fluxo,
  cfop_flags_derivados.fluxo_sentido,
  cfop_flags_derivados.fluxo_abrangencia,
  cfop_flags_derivados.codigo_operacao,
  cfop_flags_derivados.vigencia_inicio,
  cfop_flags_derivados.vigencia_fim,
  cfop_flags_derivados.indNFe,
  cfop_flags_derivados.indComunica,
  cfop_flags_derivados.indTransp,
  cfop_flags_derivados.indDevol,
  cfop_flags_derivados.indRetor,
  cfop_flags_derivados.indAnula,
  cfop_flags_derivados.indRemes,
  cfop_flags_derivados.indComb,
  cfop_flags_derivados.indTrib,
  cfop_flags_derivados.indFBKF,
  (cfop_texto.descricao ILIKE '%uso ou consumo%') AS indConsumoFinal
FROM cfop_texto
JOIN cfop_flags_derivados USING (cfop)
ORDER BY cfop;
           """
)

In [30]:
db.query("FROM mgi.cfop").df()


,cfop,DESCRICAO,APLICACAO,codigo_fluxo,fluxo_sentido,fluxo_abrangencia,codigo_operacao,vigencia_inicio,vigencia_fim,indNFe,indComunica,indTransp,indDevol,indRetor,indAnula,indRemes,indComb,indTrib,indFBKF,indConsumoFinal
0,1101,COMPRAS PARA INDUSTRIALIZAÇÃO,Classificam-se neste código as compras de merc...,1,entrada,intra-estadual,101,2006-01-01,NaN,True,False,False,False,False,False,False,False,False,False,False
1,1102,COMPRAS PARA COMERCIALIZAÇÃO,Classificam-se neste código as compras de merc...,1,entrada,intra-estadual,102,2006-01-01,NaN,True,False,False,False,False,False,False,False,False,False,False
2,1111,COMPRAS PARA INDUSTRIALIZAÇÃO DE MERCADORIA RE...,Classificam-se neste código as compras efetiva...,1,entrada,intra-estadual,111,2006-01-01,NaN,True,False,False,False,False,False,False,False,False,False,False
3,1113,COMPRAS PARA COMERCIALIZAÇÃO DE MERCADORIA REC...,Classificam-se neste código as compras efetiva...,1,entrada,intra-estadual,113,2006-01-01,NaN,True,False,False,False,False,False,False,False,False,False,False
4,1116,COMPRAS PARA INDUSTRIALIZAÇÃO OU PRODUÇÃO RURA...,Classificam-se neste código as compras de merc...,1,entrada,intra-estadual,116,2006-01-01,NaN,True,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,7556,Devolução de compra de material de uso ou consumo,Classificam-se neste código as devoluções de m...,7,saida,internacional,556,2006-01-01,NaN,True,False,False,True,False,False,False,False,False,False,True
538,7651,Venda de combustível ou lubrificante de produç...,Classificam-se neste código as vendas de combu...,7,saida,internacional,651,2006-01-01,NaN,True,False,False,False,False,False,False,True,False,False,False
539,7654,Venda de combustível ou lubrificante adquirido...,Classificam-se neste código as vendas de combu...,7,saida,internacional,654,2006-01-01,NaN,True,False,False,False,False,False,False,True,False,False,False
540,7930,Lançamento efetuado a título de devolução de b...,Classificam-se neste código os lançamentos efe...,7,saida,internacional,930,2006-01-01,NaN,True,False,False,False,False,False,False,False,False,False,False


In [31]:
# db.execute("""
#            FROM mgi.itens left join mgi.cfop
#            ON mgi.itens.cfop = mgi.cfop.cfop
           
#            """).df()

In [32]:
db.query("select count( distinct cfop) cfops_desconhecidos from mgi.itens where mgi.itens.cfop::varchar not in (select cfop::varchar from mgi.cfop)").df()

,cfops_desconhecidos
0,46


### Agrupando

In [33]:
display(db.query("select count(*) as itens from mgi.itens").df().style.format(underscore_fmt))

,itens
0,18_021_620


Há 18 milhões de itens

In [34]:
db.execute("DESCRIBE mgi.itens").df()

,column_name,column_type,null,key,default,extra
0,chave_acesso,VARCHAR,YES,None,None,None
1,timestamp_emissao,TIMESTAMP,YES,None,None,None
2,data_emissao,DATE,YES,None,None,None
3,mes_emissao,INTEGER,YES,None,None,None
4,ano_emissao,INTEGER,YES,None,None,None
5,uf_emitente,"ENUM('AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF',...",YES,None,None,None
6,uf_destinatario,"ENUM('AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF',...",YES,None,None,None
7,emitente,VARCHAR,YES,None,None,None
8,nome_emitente,VARCHAR,YES,None,None,None
9,destinatario,VARCHAR,YES,None,None,None


In [39]:
db.execute("""
            create or replace table raw.agrupamentos as
            select 
                ano_emissao as ano, mes_emissao as mes, week(timestamp_emissao) as semana, uf_emitente, uf_destinatario, 
                CASE
                    WHEN codigo_orgao_superior_destinatario = 26000 THEN 'educacao'
                    WHEN codigo_orgao_superior_destinatario = 36000 THEN 'saude'
                    WHEN codigo_orgao_superior_destinatario = -1 THEN 'sem_informacao'
                    ELSE 'administracao'
                END as categoria_destinatario,               
                ncm, descricao_ncm, cfop, sum(valor_item) as valor
            from mgi.itens
            group by all order by all
           """).df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Count
0,4917913


Reduzimos para 4.9 milhões de agrupamentos

In [40]:
db.query("from raw.agrupamentos").df()

,ano,mes,semana,uf_emitente,uf_destinatario,categoria_destinatario,ncm,descricao_ncm,cfop,valor
0,2022,1,1,AC,AC,educacao,19019090,"Outras preparações alimentícias de farinhas, e...",5405,23.37
1,2022,1,1,AC,AC,educacao,20098990,"Sucos (sumo) de outras frutas, não fermentado,...",5102,23.12
2,2022,1,1,AC,AC,educacao,2013000,"Carnes desossadas de bovino, frescas ou refrig...",5405,197.44
3,2022,1,1,AC,AC,educacao,2071400,"Pedaços e miudezas, comestíveis de galos/galin...",5405,79.56
4,2022,1,1,AC,AC,educacao,27101259,"Outras gasolinas, exceto para aviação",5929,2_052.96
...,...,...,...,...,...,...,...,...,...,...
4917908,2024,12,52,TO,TO,sem_informacao,87089529,Outras partes e acessórios de carrocerias para...,5405,210.00
4917909,2024,12,52,TO,TO,sem_informacao,87089990,Outras partes e acessórios para tratores e veí...,5403,5_140.82
4917910,2024,12,52,TO,TO,sem_informacao,87089990,Outras partes e acessórios para tratores e veí...,5405,860.00
4917911,2024,12,52,TO,TO,sem_informacao,9012100,"Café torrado, não descafeinado",5405,2_064.00


In [ ]:
db.execute("""
            CREATE OR REPLACE TABLE raw.agrupamentos_cfop AS
            SELECT DISTINCT a.cfop::varchar as cfop,
            coalesce(lower(c.DESCRICAO), 'cfop desconhecido') as descricao,
            coalesce(fluxo_sentido, CASE LEFT(a.cfop::varchar, 1)
                WHEN '1' THEN 'entrada'
                WHEN '2' THEN 'entrada'
                WHEN '3' THEN 'entrada'
                WHEN '5' THEN 'saída'
                WHEN '6' THEN 'saída'
                WHEN '7' THEN 'saída'
                ELSE 'desconhecido'
            END) as fluxo_sentido,
            coalesce(fluxo_abrangencia, case LEFT(a.cfop::varchar, 1)
                WHEN '1' THEN 'intra-estadual'
                WHEN '2' THEN 'interestadual'
                WHEN '3' THEN 'internacional'
                WHEN '5' THEN 'intra-estadual'
                WHEN '6' THEN 'interestadual'
                WHEN '7' THEN 'internacional'
                ELSE 'desconhecida'
            END) as fluxo_abrangencia,
            -- coalesce(codigo_operacao, RIGHT(a.cfop::varchar,3)) as codigo_operacao,
            -- coalesce(c.indNFe, false) as indNFe,
            -- coalesce(c.indComunica, false) as indComunica,
            -- coalesce(c.indTransp, false) as indTransp,
            -- coalesce(c.indDevol, false) as indDevol,
            -- coalesce(c.indRetor, false) as indRetor,
            -- coalesce(c.indAnula, false) as indAnula,
            -- coalesce(c.indRemes, false) as indRemes,
            -- coalesce(c.indComb, false) as indComb,
            -- coalesce(c.indTrib, false) as indTrib,
            -- coalesce(c.indFBKF, false) as indFBKF,
            -- coalesce(c.indConsumoFinal, false) as indConsumoFinal,
            CASE
                -- 1) ZERAR quando não há efeito econômico final
                WHEN indAnula or indRemes or indRetor or indTrib THEN 0
                -- 2) Sinal base por entrada/saída (1-3 entrada; 5-7 saída)
                WHEN LEFT(c.cfop::VARCHAR, 1) IN ('1','2','3')
                    THEN CASE WHEN indDevol THEN +1  ELSE -1 END   -- entrada: base -1, devolução inverte
                WHEN LEFT(c.cfop::VARCHAR, 1) IN ('5','6','7')
                    THEN CASE WHEN indDevol THEN -1  ELSE +1 END   -- saída: base +1, devolução inverte

                ELSE 1  -- defensivo para códigos atípicos
            END AS multiplicador,
            CASE
                 WHEN indAnula or indRemes or indRetor or indTrib THEN 'registro'
                 WHEN indFBKF THEN 'ffbk'
                 WHEN indConsumoFinal THEN 'consumo_final'
                 ELSE 'consumo_intermediario'
            END AS tipo_cfop 
           from raw.agrupamentos a left join mgi.cfop c on a.cfop::varchar = c.cfop::varchar order by cfop""").df()

,Count
0,406


In [89]:
db.query("select cfop, descricao from mgi.cfop where indFBKF").df()

,cfop,DESCRICAO
0,1551,COMPRA DE BEM PARA O ATIVO IMOBILIZADO
1,1552,TRANSFERÊNCIA DE BEM DO ATIVO IMOBILIZADO
2,1553,DEVOLUÇÃO DE VENDA DE BEM DO ATIVO IMOBILIZADO
3,1554,RETORNO DE BEM DO ATIVO IMOBILIZADO REMETIDO P...
4,1555,ENTRADA DE BEM DO ATIVO IMOBILIZADO DE TERCEIR...
5,2551,COMPRA DE BEM PARA O ATIVO IMOBILIZADO
6,2552,TRANSFERÊNCIA DE BEM DO ATIVO IMOBILIZADO
7,2553,DEVOLUÇÃO DE COMPRA DE BEM PARA O ATIVO IMOBIL...
8,2554,RETORNO DE BEM DO ATIVO IMOBILIZADO REMETIDO P...
9,2555,ENTRADA DE BEM DO ATIVO IMOBILIZADO DE TERCEIR...


In [82]:
db.query("from raw.agrupamentos_cfop").df()

,cfop,descricao,fluxo_sentido,fluxo_abrangencia,multiplicador,tipo_cfop
0,1101,compras para industrialização,entrada,intra-estadual,-1,consumo_intermediario
1,1102,compras para comercialização,entrada,intra-estadual,-1,consumo_intermediario
2,1111,compras para industrialização de mercadoria re...,entrada,intra-estadual,-1,consumo_intermediario
3,1113,compras para comercialização de mercadoria rec...,entrada,intra-estadual,-1,consumo_intermediario
4,1116,compras para industrialização ou produção rura...,entrada,intra-estadual,-1,consumo_intermediario
...,...,...,...,...,...,...
401,6934,cfop desconhecido,saída,interestadual,1,consumo_intermediario
402,6936,cfop desconhecido,saída,interestadual,1,consumo_intermediario
403,6949,outra saída de mercadoria ou prestação de serv...,saida,interestadual,1,consumo_intermediario
404,6990,cfop desconhecido,saída,interestadual,1,consumo_intermediario


In [86]:
db.execute("""
           SELECT a.ano, a.mes, a.semana, a.uf_emitente, a.uf_destinatario, a.categoria_destinatario,
           a.ncm, a.descricao_ncm as ncm_nome, a.valor as valor_absoluto, c.*, c.multiplicador * a.valor as valor
           FROM raw.agrupamentos a
           join raw.agrupamentos_cfop c on a.cfop = c.cfop order by all
           """).df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,ano,mes,semana,uf_emitente,uf_destinatario,categoria_destinatario,ncm,ncm_nome,valor_absoluto,cfop,descricao,fluxo_sentido,fluxo_abrangencia,multiplicador,tipo_cfop,valor
0,2022,1,1,AC,AC,educacao,19019090,"Outras preparações alimentícias de farinhas, e...",23.37,5405,venda de mercadoria adquirida ou recebida de t...,saida,intra-estadual,1,consumo_intermediario,23.37
1,2022,1,1,AC,AC,educacao,20098990,"Sucos (sumo) de outras frutas, não fermentado,...",23.12,5102,venda de mercadoria adquirida ou recebida de t...,saida,intra-estadual,1,consumo_intermediario,23.12
2,2022,1,1,AC,AC,educacao,2013000,"Carnes desossadas de bovino, frescas ou refrig...",197.44,5405,venda de mercadoria adquirida ou recebida de t...,saida,intra-estadual,1,consumo_intermediario,197.44
3,2022,1,1,AC,AC,educacao,2071400,"Pedaços e miudezas, comestíveis de galos/galin...",79.56,5405,venda de mercadoria adquirida ou recebida de t...,saida,intra-estadual,1,consumo_intermediario,79.56
4,2022,1,1,AC,AC,educacao,27101259,"Outras gasolinas, exceto para aviação",2_052.96,5929,lançamento efetuado em decorrência de emissão ...,saida,intra-estadual,1,consumo_intermediario,2_052.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4917908,2024,12,52,TO,TO,sem_informacao,87089529,Outras partes e acessórios de carrocerias para...,210.00,5405,venda de mercadoria adquirida ou recebida de t...,saida,intra-estadual,1,consumo_intermediario,210.00
4917909,2024,12,52,TO,TO,sem_informacao,87089990,Outras partes e acessórios para tratores e veí...,860.00,5405,venda de mercadoria adquirida ou recebida de t...,saida,intra-estadual,1,consumo_intermediario,860.00
4917910,2024,12,52,TO,TO,sem_informacao,87089990,Outras partes e acessórios para tratores e veí...,5_140.82,5403,venda de mercadoria adquirida ou recebida de t...,saida,intra-estadual,1,consumo_intermediario,5_140.82
4917911,2024,12,52,TO,TO,sem_informacao,9012100,"Café torrado, não descafeinado",2_064.00,5405,venda de mercadoria adquirida ou recebida de t...,saida,intra-estadual,1,consumo_intermediario,2_064.00


In [87]:
db.query("select * from mgi.de_ncm_para_scn").df()

,NCM_COD,NCM_NOME,SCN128_COD,SCN128_DESC
0,01011010,Cavalos reprodutores de raça pura,01921,"Bovinos e outros animais vivos, prods. animal,..."
1,01011090,"Animais da espécie asinina/muar, reprodutores,...",01921,"Bovinos e outros animais vivos, prods. animal,..."
2,01011100,"Cavalos reprodutores, de raça pura",01921,"Bovinos e outros animais vivos, prods. animal,..."
3,01011900,"Outros cavalos, vivos",01921,"Bovinos e outros animais vivos, prods. animal,..."
4,01012000,Asininos e muares vivos,01921,"Bovinos e outros animais vivos, prods. animal,..."
...,...,...,...,...
11870,99997101,Pedras em bruto do capítulo 71 da NCM,05802,Minerais não-metálicos
11871,99997102,Pedras lapidadas/trabalhadas do capítulo 71 da...,31802,Produtos de industrias diversas
11872,99997103,Joalheria de ouro do capitulo 71 da NCM,31802,Produtos de industrias diversas
11873,99997104,Outros artigos do capitulo 71 da NCM,31802,Produtos de industrias diversas


In [ ]:
db.execute(""" 
           SELECT a.ano, a.mes, a.semana, a.uf_emitente, a.uf_destinatario, a.categoria_destinatario, a.ncm, a.ncm_nome, 
           """)

In [ ]:
db.execute("create or replace table mgi.cnpjs_uniao as select * from read_parquet('../data/inputs/rfb_cnpj/cnpjs_uniao.parquet')")

In [ ]:
db.query("""select list(distinct cnpj) from mgi.cnpjs_uniao where tipo_atividade != 'educacao' and atividade_economica in (
    'Atividades de apoio à educação, exceto caixas escolares',
    'Atividades de bibliotecas e arquivos',
    'Comércio varejista de livros',
    'Edição de livros',
    'Edição integrada à impressão de livros',
    'Educação profissional de nível tecnológico',
    'Educação profissional de nível técnico',
    'Educação superior - graduação',
    'Educação superior - graduação e pós-graduação',
    'Educação superior - pós-graduação e extensão',
    'Ensino fundamental',
    'Ensino médio',
    'Impressão de livros, revistas e outras publicações periódicas',
    'Outras atividades de ensino não especificadas anteriormente'
)""")

In [ ]:
db.execute("SELECT DISTINCT codigo_orgao_superior_destinatario, orgao_superior_destinatario FROM nfe.cgu.notas order by orgao_superior_destinatario").df()

In [ ]:
# # raw.compras_detalhadas
# chave_acesso, timestamp_emissao, data_emissao, emitente, nome_emitente, destinatario, nome_destinatario, codigo_orgao_destinatario, codigo_orgao_superior_destinatario, numero_produto, descricao_produto, ncm, descricao_ncm, scn128, produto, scn68, atividade, cfop, descricao_cfop, valor_item, unidade_medida, quantidade, valor_unitario, cfop, categoria_cfop, multiplicador, valor_item



In [ ]:
db.query("""
SELECT
    COUNT(*) FILTER (WHERE c.fluxo_sentido = 'saida') * 100.0 / COUNT(*) AS pct_saida
FROM mgi.itens i
JOIN mgi.cfop c 
    ON i.cfop::VARCHAR = c.cfop
-- WHERE indDevol
""").df()

In [ ]:
#  